In [1]:
try:
    import scienceplots
    import rootutils
except ModuleNotFoundError:
    !pip install SciencePlots
    !pip install rootutils

  Using cached SciencePlots-2.1.1-py3-none-any.whl.metadata (11 kB)
Using cached SciencePlots-2.1.1-py3-none-any.whl (16 kB)


In [1]:
import shutup
shutup.please()

import os
os.environ['CUDA_VISIBLE_DEVICES']='1'

import warnings
warnings.filterwarnings('ignore')

import rootutils
ROOT = rootutils.setup_root(indicator='README.md', search_from=os.path.abspath(''), pythonpath=True, cwd=True)

import jax
import jax.numpy as jnp
GLOBAL_KEY = jax.random.key(42)

import matplotlib.pyplot as plt
import scienceplots
plt.style.use(['science', 'notebook'])